# Deimos Router

Deimos is the smallest, ugliest moon of Mars.

Deimos is also the smallest, dumbest way to do routing.

In [1]:
import deimos_router

In [21]:
from deimos_router import Router, register_router, chat
from deimos_router.rules import TaskRule, AutoTaskRule, CodeRule, CodeLanguageRule, NaturalLanguageRule, MessageLengthRule, ConversationContextRule, Rule, Decision

## Basic Usage - Task Based Routing

A Router uses one or more Rules to select a model. The simplest Rule is TaskRule, which maps task names to models.

In [3]:
Router(
   name="my-first-router",
   rules=[
       TaskRule(
           name="task-based-routing",
           triggers={
               'coding': 'openai/gpt-5',
               'creative': 'openai/gpt-4o',
               'simple': 'openai/gpt-5-nano'
           }
       )
   ],
   default="openai/gpt-4o-mini"
)



Router(name='my-first-router', models=[])

To use a Router, make a call to `chat.completions.create`, as if it were a call to OpenAI's SDK, and specify the router as the model.

For simple task-based routing, include the task name as an argument.

In [4]:
# Use the router for chat completions
response = chat.completions.create(
   model="deimos/my-first-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response.choices[0].message.content)

def fibonacci(n: int) -> int:
    """
    Return the nth Fibonacci number with F(0)=0 and F(1)=1.
    Uses fast doubling (O(log n)). n must be a non-negative integer.
    """
    if n < 0:
        raise ValueError("n must be non-negative")

    def fib_pair(k: int) -> tuple[int, int]:
        if k == 0:
            return (0, 1)
        a, b = fib_pair(k // 2)
        c = a * (2 * b - a)      # F(2m)
        d = a * a + b * b        # F(2m+1)
        return (c, d) if k % 2 == 0 else (d, c + d)

    return fib_pair(n)[0]


Details about the routing decision can be found in the response at `._deimos_metadata`.

In [5]:
response._deimos_metadata

{'router_used': 'my-first-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'TaskRule',
   'rule_name': 'task-based-routing',
   'rule_trigger': 'coding',
   'decision': 'openai/gpt-5'}]}

## AutoTask Routing

An `AutoTaskRule` is created in the same way as a `TaskRule`, but the task is determined by a call to a small language model.

In [6]:
auto_task = AutoTaskRule(
    name = "auto-task-rule",
    triggers = {
        "creative writing" : "openai/gpt-4o",
        "writing code" : "openai/gpt-5",
        "informational" : "openai/gpt-5-mini",
        "haiku composition" : "openai/gpt-5-nano"
    },
)


        

In [7]:
auto_router = Router(
    name = "auto-router",
    rules = [auto_task],
    default="openai/gpt-4o-mini"
    )


In [8]:
# Use the router for chat completions
response1 = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a python function that finds the nth fibonacci number"}
   ],
   task="coding"
)

print(response1.choices[0].message.content)

def fib(n: int) -> int:
    """
    Return the nth Fibonacci number (0-indexed: F(0)=0, F(1)=1).
    Uses fast doubling for O(log n) time.
    """
    if not isinstance(n, int):
        raise TypeError("n must be an int")
    if n < 0:
        raise ValueError("n must be non-negative")

    def _fib(k: int) -> tuple[int, int]:
        if k == 0:
            return 0, 1
        a, b = _fib(k // 2)
        c = a * (2 * b - a)
        d = a * a + b * b
        if k % 2 == 0:
            return c, d
        else:
            return d, c + d

    return _fib(n)[0]


In [9]:
response1._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'writing code',
   'decision': 'openai/gpt-5'}]}

In [10]:
haiku = chat.completions.create(
   model="deimos/auto-router",
   messages=[
       {"role": "user", "content": "Write a short japanese poem about mars, and follow the syllable count 5, 7, 5"}
   ],
)

print(haiku.choices[0].message.content)

あかいほし
砂を吹く風
あかいゆめ


In [11]:
haiku._deimos_metadata

{'router_used': 'auto-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'available_models': [],
 'explain': [{'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'haiku composition',
   'decision': 'openai/gpt-5-nano'}]}

## Code / Not code Routing

The `CodeRule` is a very simple rule that determines whether a prompt contains code and routes based on that.

In [12]:
code_or_not = CodeRule(
    name = "code-or-not-code",
    code = "openai/gpt-5",
    not_code = "openai/gpt-4o"
)

code_no_code_router = Router(
    name = "code-nocode-router",
    rules = [code_or_not]
)



In [13]:
prompt = """
Debug this:
```
def multiply(x, y):
    return x**y
```
"""


might_be_code_1 = chat.completions.create(
   model="deimos/code-nocode-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(might_be_code_1.choices[0].message.content)

You used exponentiation (**) instead of multiplication. Here’s the fix:

```python
def multiply(x, y):
    return x * y
```

Quick check:
- multiply(2, 3) -> 6
- multiply(-4, 5) -> -20
- multiply(3.5, 2) -> 7.0


In [14]:
might_be_code_1._deimos_metadata

{'router_used': 'code-nocode-router',
 'selected_model': 'openai/gpt-5',
 'original_model_field': 'openai/gpt-5',
 'available_models': [],
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'code_detected',
   'decision': 'openai/gpt-5'}]}

In [15]:
prompt = """
What is the smallest, ugliest moon of Mars?
"""


might_be_code_2 = chat.completions.create(
   model="deimos/code-nocode-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(might_be_code_2.choices[0].message.content)

Mars has two small moons, Phobos and Deimos. Of the two, Phobos is considered both the smaller and the more irregularly shaped, often described as the less aesthetically pleasing or "uglier" moon. Phobos is heavily cratered with a somewhat lumpy appearance. It is also the closer of the two moons to Mars and is gradually spiraling inward, which will eventually lead to it either crashing into Mars or breaking apart.


In [16]:
might_be_code_2._deimos_metadata

{'router_used': 'code-nocode-router',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'available_models': [],
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'no_code_detected',
   'decision': 'openai/gpt-4o'}]}

## Code Language Routing

To route based on programming language, use a CodeLanguage rule.

This rule uses regex to classify among several popular languages, and then falls back to a small language model call to determine language.

In [17]:
code_lang_rule = CodeLanguageRule(
    name = "code-lang-rule",
    language_mappings = {
        "python" : "openai/gpt-3.5-turbo",
        "sql" : "openai/gpt-5-mini",
    }
)

Router(
    name = "code-lang-router",
    rules = [code_lang_rule]
)


Router(name='code-lang-router', models=[])

In [18]:
prompt = """
What does this function do?
```
def _idk(x):
    if n < 0:
        raise ValueError("n must be non-negative")
    a, b = 0, 1  # (F(m), F(m+1))
    for bit in bin(n)[2:]:
        c = a * (2 * b - a)     # F(2m)
        d = a * a + b * b       # F(2m+1)
        if bit == '0':
            a, b = c, d
        else:
            a, b = d, c + d
    return a
```
"""

what_it_do = chat.completions.create(
   model="deimos/code-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(what_it_do.choices[0].message.content)

This function calculates the nth Fibonacci number iteratively using the binary representation of the input number `n`. It computes Fibonacci numbers based on the properties of the Fibonacci sequence and the binary representation of the input number to optimize the calculation process.


In [19]:
what_it_do._deimos_metadata

{'router_used': 'code-lang-router',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'available_models': [],
 'explain': [{'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'python',
   'decision': 'openai/gpt-3.5-turbo'}]}

In [20]:
prompt = """
What does code do?
```
SELECT *
FROM people
WHERE name = "john"
```
"""

what_it_do = chat.completions.create(
   model="deimos/code-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(what_it_do.choices[0].message.content)

This is an SQL query that returns all rows from the table named "people" where the name column equals "john".

Breakdown:
- SELECT * — return all columns.
- FROM people — from the table named people.
- WHERE name = "john" — only include rows whose name equals john.

Notes and caveats:
- Standard SQL uses single quotes for string literals: WHERE name = 'john'. Some databases (e.g. MySQL with certain modes) accept double quotes for strings, while others (e.g. PostgreSQL) treat double quotes as identifiers.
- Comparison case-sensitivity depends on the database collation. To match regardless of case you can use LOWER(name) = 'john' or ILIKE 'john' (Postgres).
- For safety when plugging user input into this query, use parameterized queries/prepared statements to avoid SQL injection.
- If there is an index on name, the filter can be faster.


In [21]:
what_it_do._deimos_metadata

{'router_used': 'code-lang-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'available_models': [],
 'explain': [{'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'sql',
   'decision': 'openai/gpt-5-mini'}]}

## NaturalLanguageRule

To route based on the language of the request (English, French, Spanish, etc), use a NaturalLanguageRule. This calls a small language model to detect the language. Specify language:model mapping using the two-letter ISO language code (`EN`, `FR`, `ES`).

In [22]:
nat_lang_rule = NaturalLanguageRule(
    name = "nat-lang-rule",
    language_mappings = {
        "EN" : "openai/gpt-3.5-turbo",
        "ES" : "openai/gpt-5-mini",
    }
)

Router(
    name = "nat-lang-router",
    rules = [nat_lang_rule]
)


Router(name='nat-lang-router', models=[])

In [23]:
prompt = """
Tell me a joke about language.
"""

lang_joke = chat.completions.create(
   model="deimos/nat-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(lang_joke.choices[0].message.content)

Why did the grammar teacher go to jail? 

For excessive use of sentences!


In [24]:
lang_joke._deimos_metadata

{'router_used': 'nat-lang-router',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'available_models': [],
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'EN',
   'decision': 'openai/gpt-3.5-turbo'}]}

In [25]:
prompt = """
Cuéntame un chiste sobre el lenguaje.
"""

lang_joke = chat.completions.create(
   model="deimos/nat-lang-router",
   messages=[
       {"role": "user", "content": prompt}
   ],
)

print(lang_joke.choices[0].message.content)

¿Sabes por qué el verbo fue al psicólogo?
—Porque no podía conjugar sus sentimientos. 

¿Quieres otro chiste lingüístico?


In [26]:
lang_joke._deimos_metadata

{'router_used': 'nat-lang-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'available_models': [],
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'ES',
   'decision': 'openai/gpt-5-mini'}]}

## MessageLengthRule


The MessageLengthRule selects one of three models based on length (in tokens) of the user message. 

- short: below the `short_threshold`
- medium: between the `short_threshold` and the `long_threshold`
- long: above the `long_threshold`

In [3]:
len_rule = MessageLengthRule(
    name = "len_rule",
    short_threshold = 50,
    long_threshold = 200,
    short_model = "openai/gpt-5-nano",
    medium_model = "openai/gpt-5-mini",
    long_model = "openai/gpt-5"
)

Router(
    name = "length-router",
    rules = [len_rule],
)


Router(name='length-router', models=[])

In [4]:
short_prompt = "What color is Mars?"

short_response= chat.completions.create(
   model="deimos/length-router",
   messages=[
       {"role": "user", "content": short_prompt}
   ],
)

print(short_response.choices[0].message.content)

Mars is typically described as red or reddish-orange. This color comes from iron oxide (rust) dust on its surface and in its atmosphere. Its appearance can vary with lighting and the season, but the hallmark is the reddish hue.


In [5]:
short_response._deimos_metadata

{'router_used': 'length-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'available_models': [],
 'explain': [{'rule_type': 'MessageLengthRule',
   'rule_name': 'len_rule',
   'rule_trigger': 'short_message_5_tokens',
   'decision': 'openai/gpt-5-nano'}]}

In [7]:
medium_prompt = "Imagine you’ve just been hired as the Chief Imagination Officer for a brand-new amusement park called DreamTopia. Your first assignment is to design the park’s most unusual and delightful attraction, something that no other park has ever seen before. It should combine at least two completely different ideas (for example: a roller coaster made of books, or a water slide that doubles as a musical instrument). Please describe the attraction in detail: how it looks, how it works, and what makes it magical or fun. End with a short tagline or slogan that could go on the park’s posters."

medium_response= chat.completions.create(
   model="deimos/length-router",
   messages=[
       {"role": "user", "content": medium_prompt}
   ],
)

print(medium_response.choices[0].message.content)

Name: Dreamloom — The Coaster That Weaves Your Story

Concept in one line
A family-friendly roller-coaster crossed with a giant kinetic loom and an interactive composing studio: every ride lets guests literally add threads, music, and motifs to a communal tapestry that grows and plays the day’s unique song.

How it looks
- Exterior: a low, graceful building shaped like an oversized shuttle loom — curved wooden ribs, giant bobbins that turn, and glowing fiber-optic “threads” stretching up the facade. At night the strands pulse like constellations.
- Interior: the track loops around a cathedral-scale vertical loom called the Loom of Lumin (a 40–60 ft wall of warp threads). Suspended balconies and catwalks hold spindles, gears, and artisans’ lamps. Projection-mapped murals shift between four themed “chapters” (Forest, Ocean, Sky, City) around the loom.
- Cars: the coaster cars are shuttle-shaped benches with a tactile “Spindle Console” in front of each rider — a small, kid-friendly panel 

In [8]:
medium_response._deimos_metadata

{'router_used': 'length-router',
 'selected_model': 'openai/gpt-5-mini',
 'original_model_field': 'openai/gpt-5-mini',
 'available_models': [],
 'explain': [{'rule_type': 'MessageLengthRule',
   'rule_name': 'len_rule',
   'rule_trigger': 'medium_message_119_tokens',
   'decision': 'openai/gpt-5-mini'}]}

## ConversationContextRule

ConversationContextRule selects model based on the number of back-and-forth messages in a conversation, classifying a conversation as new, developing, or deep.

- new conversation: less than `new_threshold` number of messages
- developing conversation: between `new_threshold` and `deep_theshold` number of messages
- deep conversation: longer than `deep_threshold` number of messages

In [12]:
context_rule = ConversationContextRule(
    name = "conv-context-rule",
    new_threshold = 3,
    deep_threshold = 10,
    new_model = "openai/gpt-5-nano",
    developing_model = "openai/gpt-4o-mini",
    deep_model = "openai/gpt-4o"
)

Router(
    name = "context-router",
    rules = [context_rule]
)

Router(name='context-router', models=[])

In [15]:
new_conversation= chat.completions.create(
   model="deimos/context-router",
   messages=[
       {"role": "user", "content": "Tell me the funnest fact about Deimos (the moon, not the god)."}
   ],
)

print(new_conversation.choices[0].message.content)

From Deimos, Mars would dominate the sky: Mars would look about 16 degrees across (roughly 32 times bigger than the Moon looks from Earth). So you’d see a gigantic, orange Mars hanging in your sky if you stood on that tiny moon.

Quick extra bits:
- Deimos is tiny—only about 12 km across.
- It’s thought to be a captured asteroid, not formed around Mars.


In [16]:
new_conversation._deimos_metadata

{'router_used': 'context-router',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'available_models': [],
 'explain': [{'rule_type': 'ConversationContextRule',
   'rule_name': 'conv-context-rule',
   'rule_trigger': 'new_conversation_1_messages_62_chars',
   'decision': 'openai/gpt-5-nano'}]}

In [17]:
conversation = [
    {
      "role": "user",
      "content": "Hi there! Can you help me come up with a fun name for a coffee shop?"
    },
    {
      "role": "assistant",
      "content": "Of course! How about 'Bean There, Done That'?"
    },
    {
      "role": "user",
      "content": "Haha, that’s clever. Can you give me a couple more options?"
    },
    {
      "role": "assistant",
      "content": "Sure! Some other ideas: 'Daily Grind Café' and 'Perk Up Coffeehouse.'"
    },
    {
      "role": "user",
      "content": "Nice! I like 'Perk Up.' Can you suggest a tagline to go with it?"
    },
    {
      "role": "assistant",
      "content": "How about: 'Perk Up — Where Every Cup Sparks Joy'?"
    },
    {
      "role": "user",
      "content": "Perfect, that’s exactly what I was looking for. Thanks!"
    }
  ]



In [18]:
developing_conversation= chat.completions.create(
   model="deimos/context-router",
   messages=conversation,
)

print(developing_conversation.choices[0].message.content)

You're welcome! I'm glad you liked it. If you need any more help or ideas, feel free to ask. Good luck with your coffee shop!


In [19]:
developing_conversation._deimos_metadata

{'router_used': 'context-router',
 'selected_model': 'openai/gpt-4o-mini',
 'original_model_field': 'openai/gpt-4o-mini',
 'available_models': [],
 'explain': [{'rule_type': 'ConversationContextRule',
   'rule_name': 'conv-context-rule',
   'rule_trigger': 'developing_conversation_7_messages_410_chars',
   'decision': 'openai/gpt-4o-mini'}]}

## Custom Rule Types

You can create your own custom Rule type by subclassing Rule and implementing `evaluate` with the following signature:

```
def evaluate(self, request_data: Dict[str, Any]) -> Decision:
    # return a Decision
```

A `Decision` has two arguments, a model str and a trigger str. The trigger str explains why the decision was made.

In [27]:
import random

class RandomRule(Rule):
    """Selects a model at random."""

    def __init__(self, name:str, models: list[str]):
        super().__init__(name)
        
        self.models = models

    def evaluate(self, _) -> Decision:
        model = random.choice(self.models)
        return Decision(model, "random_selection")
        

In [28]:
rand_rule = RandomRule(
    name = "rand-rule",
    models = [
        "openai/gpt-5-nano",
        "qwen/qwen-turbo",
        "x-ai/grok-3-mini"
    ]
)

Router(
    name = "random-router",
    rules = [rand_rule]
)

Router(name='random-router', models=[])

In [30]:
for _ in range(5):

    prompt = "Tell me something about yourself."

    response= chat.completions.create(
       model="deimos/random-router",
       messages=[
           {"role": "user", "content": prompt}
       ],
    )
    
    print(response.choices[0].message.content)
    print()
    print(response._deimos_metadata)
    print("---")

I’m ChatGPT, an AI assistant created by OpenAI. Here’s a quick snapshot:

- I can help with writing, editing, brainstorming, explaining concepts, summarizing, translating, coding, debugging, planning, and more.
- I can adapt my tone and style to be formal, casual, concise, or imaginative—whatever fits your goal.
- I have knowledge up to June 2024 and don’t browse real-time unless a tool is enabled. Some information may be outdated, and I can help verify details or guide you to reliable sources.
- I don’t know your personal data unless you share it here, and I don’t retain memory of our chats across sessions unless a memory feature is used.
- I generate responses based on patterns in data, not from personal beliefs or experiences, and I can explain my reasoning steps if you’d like a transparent answer (though I can also provide direct results if you prefer).

What would you like to work on today?

{'router_used': 'random-router', 'selected_model': 'openai/gpt-5-nano', 'original_model_fi

## Rule Chaining

Rules can call other rules instead of models. For example, you might want to first determine if code is present before determining which code language is being used.

In [32]:
code_lang_rule = CodeLanguageRule(
    name = "code-lang-rule",
    language_mappings = {
        "python" : "openai/gpt-3.5-turbo",
        "sql" : "openai/gpt-5-mini",
    }
)

code_or_not = CodeRule(
    name = "code-or-not-code",
    code = code_lang_rule,
    not_code = "openai/gpt-4o"
)



Router(
    name = "maybe_code_maybe_not",
    rules = [code_or_not]
)


Router(name='maybe_code_maybe_not', models=[])

In [36]:
not_code_prompt = "Tell me a joke."

not_code_response= chat.completions.create(
   model="deimos/maybe_code_maybe_not",
   messages=[
       {"role": "user", "content": not_code_prompt}
   ],
)

print(not_code_response.choices[0].message.content)

Why don't skeletons fight each other?

They don't have the guts!


In [37]:
not_code_response._deimos_metadata

{'router_used': 'maybe_code_maybe_not',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'available_models': [],
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'no_code_detected',
   'decision': 'openai/gpt-4o'}]}

In [38]:
code_prompt = """
What does this function do?
```
def _idk(x):
    if n < 0:
        raise ValueError("n must be non-negative")
    a, b = 0, 1  # (F(m), F(m+1))
    for bit in bin(n)[2:]:
        c = a * (2 * b - a)     # F(2m)
        d = a * a + b * b       # F(2m+1)
        if bit == '0':
            a, b = c, d
        else:
            a, b = d, c + d
    return a
```
"""

code_response = chat.completions.create(
   model="deimos/maybe_code_maybe_not",
   messages=[
       {"role": "user", "content": code_prompt}
   ],
)

print(code_response.choices[0].message.content)

This function calculates the nth Fibonacci number using an iterative method that computes the Fibonacci numbers in a binary representation order. It iterates through the binary representation of the input number `n`, and at each step calculates the next Fibonacci numbers based on whether the current bit is 0 or 1. The function then returns the nth Fibonacci number.


In [39]:
code_response._deimos_metadata

{'router_used': 'maybe_code_maybe_not',
 'selected_model': 'openai/gpt-3.5-turbo',
 'original_model_field': 'openai/gpt-3.5-turbo',
 'available_models': [],
 'explain': [{'rule_type': 'CodeRule',
   'rule_name': 'code-or-not-code',
   'rule_trigger': 'code_detected',
   'decision': 'continue'},
  {'rule_type': 'CodeLanguageRule',
   'rule_name': 'code-lang-rule',
   'rule_trigger': 'python',
   'decision': 'openai/gpt-3.5-turbo'}]}

## Rule Fallthrough

You can also list more than one rule in the router. In this case, if the first rule does not match any defined case, the next rule will be used, and so on until a rule that returns a model is found. You can also define a default model on a router, which is used if no rule returns a model.

In [40]:
nat_lang_rule = NaturalLanguageRule(
    name = "nat-lang-rule",
    language_mappings = {
        "FR" : "openai/gpt-3.5-turbo",
        "ES" : "openai/gpt-5-mini",
    }
)

auto_task = AutoTaskRule(
    name = "auto-task-rule",
    triggers = {
        "writing code" : "openai/gpt-5",
        "medical advice" : "openai/gpt-5-mini",
        "haiku composition" : "openai/gpt-5-nano"
    },
)

Router(
    name = "haiku-but-not-limericks",
    rules = [nat_lang_rule, auto_task],
    default = "openai/gpt-4o"
)

Router(name='haiku-but-not-limericks', models=[])

In [42]:
haiku_request = "Write me a short poem with three lines and 17 syllables on Deimos (the god, not the moon)."

haiku_response = chat.completions.create(
   model="deimos/haiku-but-not-limericks",
   messages=[
       {"role": "user", "content": haiku_request}
   ],
)

print(haiku_response.choices[0].message.content)

Deimos, god of dread
in war's tremor, he smiles.
for courage, fear's lone twin


In [43]:
haiku_response._deimos_metadata

{'router_used': 'haiku-but-not-limericks',
 'selected_model': 'openai/gpt-5-nano',
 'original_model_field': 'openai/gpt-5-nano',
 'available_models': [],
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'no_language_detected',
   'decision': 'no_match'},
  {'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'haiku composition',
   'decision': 'openai/gpt-5-nano'}]}

In [45]:
limerick_request = "Write a funny five line poem, with AABBA rhyme scheme and a sing-songy meter, about any computer science subject."

limerick_response = chat.completions.create(
   model="deimos/haiku-but-not-limericks",
   messages=[
       {"role": "user", "content": limerick_request}
   ],
)

print(limerick_response.choices[0].message.content)

In a world of loops and arrays,  
A coder got lost in a daze.  
He debugged through the night,  
Thinking all was just right,  
Till he printed "ten" stars in a blaze.


In [46]:
limerick_response._deimos_metadata

{'router_used': 'haiku-but-not-limericks',
 'selected_model': 'openai/gpt-4o',
 'original_model_field': 'openai/gpt-4o',
 'available_models': [],
 'explain': [{'rule_type': 'NaturalLanguageRule',
   'rule_name': 'nat-lang-rule',
   'rule_trigger': 'no_language_detected',
   'decision': 'no_match'},
  {'rule_type': 'AutoTaskRule',
   'rule_name': 'auto-task-rule',
   'rule_trigger': 'None',
   'decision': 'no_match'},
  {'rule_type': 'default',
   'rule_name': 'default',
   'rule_trigger': 'None',
   'decision': 'openai/gpt-4o'}]}